# Refactored Mesh Generation
- In this notebook  I will go through the process of generating meshes
- I will show how to read in a `.swc` file 
- Perform refinements using `scipy` splines 
- Perform refinements by splitting edges only
- Generate trunks
- Generate Meshes

In the first cell, I load the necessary modules, this include `neuronswc.py` and `neuronmeshgenerator.py` which contain the main functionality

In [1]:
import os
import sys
print("Python version: ",sys.version)
print("Version info:   ",sys.version_info)

# add path to get to the modules renderneuron and parallelframe
sys.path.insert(0,'../src/')
import neuronswc as ns
import neuronmeshgenerator as nmg
import networkx as nx

Python version:  3.8.10 (default, Mar 13 2023, 10:26:41) 
[GCC 9.4.0]
Version info:    sys.version_info(major=3, minor=8, micro=10, releaselevel='final', serial=0)
JSON:            2.0.9
networkx:        3.1
scipy:           1.10.1


## Read in a geomtry from `.swc`
+ In this section we will read in a geometry that was downloaded from [`NeuroMorpho.org`](https://neuromorpho.org/)
+ The geometries are located in the `../cells/` folder

In [2]:
neuron_filename='../cells/0-2a.CNG.swc'
G=ns.read_1d_neuron_swc(neuron_filename)

## Edge Split Refinement
+ In this next cell we will do an edge splitting of the geometry and save the geometry as an `.swc` and `.ugx` formats
+ We will also remove the soma line from the geometry
+ We will also generate a trunk geometry where all the subsets are separated
+ I also write the output as soma and neurite only subsets

In [7]:
# remove the soma_line
G=ns.remove_soma_line(G)

# split all  the edges
Gsplit=ns.split_edges(G)

# save to .swc and .ugx formats
ns.save_1d_neuron_swc(Gsplit,'edge_split.swc')
ns.write_1d_ugx(Gsplit,'edge_split.ugx')

# get neuron trunks
trunks,T=ns.get_trunks(Gsplit)
ns.write_1d_ugx(T,'trunks.ugx')

# save only as soma and neurite geometry
ns.save_1d_neuron_swc_soma_neurite(Gsplit,'soma_neurite_split.swc')
ns.write_1d_ugx_soma_neurite(Gsplit,'soma_neurite_split.ugx')

Saved to  edge_split.swc
Saved to  edge_split.ugx
Saved to  trunks.ugx
Saved to  soma_neurite_split.swc
Saved to  soma_neurite_split.ugx
Saved to  soma_neurite_split.ugx


## Spline Refinement
+ I repeat the previous cell except we are using a spline refinement methodology

In [11]:
# Spline Refine
dx=0.25 # specify a delta x
Gspline=ns.spline_neuron(G,dx)

# save to .swc and .ugx formats
ns.save_1d_neuron_swc(Gspline,'edge_spline.swc')
ns.write_1d_ugx(Gspline,'edge_spline.ugx')

# get neuron trunks
trunks,T=ns.get_trunks(Gspline)
ns.write_1d_ugx(T,'trunks_spline.ugx')

# save only as soma and neurite geometry
ns.save_1d_neuron_swc_soma_neurite(Gspline,'soma_neurite_spline.swc')
ns.write_1d_ugx_soma_neurite(Gspline,'soma_neurite_spline.ugx')

Saved to  edge_spline.swc
Saved to  edge_spline.ugx
Saved to  trunks_spline.ugx
Saved to  soma_neurite_spline.swc
Saved to  soma_neurite_spline.ugx
Saved to  soma_neurite_spline.ugx


## Generate a Mesh
In this cell we will take our refinements and make surface meshes. I will use the `T`, `Gsplit`, and `Gspline` geometry we generated

In [16]:
n_spcontours=10
n_spcircle_points=16
n_circle_points=12
nmg.write_mesh_with_soma_sphere_ugx(Gspline,n_circle_points,n_spcontours,n_spcircle_points,'mesh_spline.ugx')
nmg.write_mesh_with_soma_sphere_ugx(Gsplit,n_circle_points,n_spcontours,n_spcircle_points,'mesh_split.ugx')
nmg.write_mesh_with_soma_sphere_ugx(T,n_circle_points,n_spcontours,n_spcircle_points,'mesh_trunks_spline.ugx')

Saved to  mesh_spline.ugx
Saved to  mesh_split.ugx
Saved to  mesh_trunks_spline.ugx


## Generate Sequence of Meshes and `.vrn`
In this cell we will use a function that I made which will generate a sequence of meshes and `.zip` into a VRN file type for use in the NeuroVisor project.

In [17]:
neuron_filename='../cells/0-2a.CNG.swc'
num_ref=10
start_dx=128.0
n_cir_points=12
sph_contours=32
sph_points=32
MESHFOLDER=nmg.make_meshes(neuron_filename,start_dx,num_ref,n_cir_points,sph_contours,sph_points)
nmg.write_vrn(MESHFOLDER)

Saved to  0-2a.CNG.mesh/0-2a.CNGrefinement0.swc
Saved to  0-2a.CNG.mesh/0-2a.CNG_1d_refinement0.ugx
Saved to  0-2a.CNG.mesh/0-2a.CNG_3d_mesh_refinment0.ugx
Saved to  0-2a.CNG.mesh/0-2a.CNGrefinement1.swc
Saved to  0-2a.CNG.mesh/0-2a.CNG_1d_refinement1.ugx
Saved to  0-2a.CNG.mesh/0-2a.CNG_3d_mesh_refinment1.ugx
Saved to  0-2a.CNG.mesh/0-2a.CNGrefinement2.swc
Saved to  0-2a.CNG.mesh/0-2a.CNG_1d_refinement2.ugx
Saved to  0-2a.CNG.mesh/0-2a.CNG_3d_mesh_refinment2.ugx
Saved to  0-2a.CNG.mesh/0-2a.CNGrefinement3.swc
Saved to  0-2a.CNG.mesh/0-2a.CNG_1d_refinement3.ugx
Saved to  0-2a.CNG.mesh/0-2a.CNG_3d_mesh_refinment3.ugx
Saved to  0-2a.CNG.mesh/0-2a.CNGrefinement4.swc
Saved to  0-2a.CNG.mesh/0-2a.CNG_1d_refinement4.ugx
Saved to  0-2a.CNG.mesh/0-2a.CNG_3d_mesh_refinment4.ugx
Saved to  0-2a.CNG.mesh/0-2a.CNGrefinement5.swc
Saved to  0-2a.CNG.mesh/0-2a.CNG_1d_refinement5.ugx
Saved to  0-2a.CNG.mesh/0-2a.CNG_3d_mesh_refinment5.ugx
Saved to  0-2a.CNG.mesh/0-2a.CNGrefinement6.swc
Saved to  0-2a.C